In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC,SVR
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score
from imblearn.under_sampling import RandomUnderSampler
from datasist.structdata import detect_outliers
from sklearn.impute import KNNImputer

In [2]:
# loading the dataset.
data = pd.read_csv('../data/processed/processed_data.csv')

In [3]:
data

,age_group,length_of_stay,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M,type_of_admission_Emergency,type_of_admission_Newborn,type_of_admission_Not Available,type_of_admission_Trauma,type_of_admission_Urgent,apr_medical_surgical_description_Surgical
0,4,5,10.0,5.0,1,0,1,0,0,0,0,0
1,2,8,10.0,5.0,2,0,1,0,0,0,0,0
2,3,8,10.0,5.0,1,1,1,0,0,0,0,0
3,4,4,3.0,5.0,1,0,1,0,0,0,0,0
4,4,3,4.0,5.0,2,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
193510,3,3,8.0,10.0,3,1,1,0,0,0,0,1
193511,5,11,10.0,8.0,2,1,1,0,0,0,0,1
193512,2,5,2.0,2.0,1,0,0,0,0,0,1,1
193513,2,13,17.0,5.0,2,0,0,0,0,0,1,0


In [4]:
#checking for duplication in data 
data.duplicated().sum()

0

In [5]:
# Geitting Information about the Data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193515 entries, 0 to 193514
Data columns (total 12 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   age_group                                  193515 non-null  int64  
 1   length_of_stay                             193515 non-null  int64  
 2   ccs_diagnosis_description                  193515 non-null  float64
 3   ccs_procedure_description                  193515 non-null  float64
 4   apr_severity_of_illness_description        193515 non-null  int64  
 5   gender_M                                   193515 non-null  int64  
 6   type_of_admission_Emergency                193515 non-null  int64  
 7   type_of_admission_Newborn                  193515 non-null  int64  
 8   type_of_admission_Not Available            193515 non-null  int64  
 9   type_of_admission_Trauma                   193515 non-null  int64  
 10  type_of_

In [6]:
# getting copy of the data set
df = data.copy()

In [7]:
df.describe()

,age_group,length_of_stay,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M,type_of_admission_Emergency,type_of_admission_Newborn,type_of_admission_Not Available,type_of_admission_Trauma,type_of_admission_Urgent,apr_medical_surgical_description_Surgical
count,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000,193515.000000
mean,3.549751,16.002217,7.505754,8.365930,2.635956,0.510121,0.541111,0.016795,0.004837,0.019812,0.183603,0.416112
std,1.300368,18.350912,3.878398,5.395328,1.018068,0.499899,0.498308,0.128501,0.069379,0.139356,0.387161,0.492914
min,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,5.000000,5.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,10.000000,7.000000,7.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,20.000000,9.000000,10.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,5.000000,120.000000,32.000000,41.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Although the size of the data after cleaning and preprocessing decreese greatly to 385,003 instance and 14 features but it is still challenging to deal with for limted pressoer laptob so we are going take small sample to run the models more efficently while maintaing the same chracterstic of the data by usin straitify method.

In [8]:
# sampled_data = df.sample(n=20000, st= df['length_of_stay'])

In [9]:
x = df.drop("length_of_stay", axis = 1)
y = df["length_of_stay"]

In [10]:
main_x,sample_x,main_y,sample_y = train_test_split(x,y,test_size=0.05,random_state=42,stratify=y)

In [11]:
x_train,x_test,y_train,y_test = train_test_split(sample_x,sample_y,test_size=0.3)

In [12]:
len(y_train)

6773

In [13]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [14]:
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)

LinearRegression()

In [15]:
print('Train Score', lr.score(x_train_scaled,y_train))
print('Test Score', lr.score(x_test_scaled,y_test))

Train Score 0.16382055728313283
Test Score 0.17735894958277587


In [16]:
knnr = KNeighborsRegressor()

In [17]:
knnr.fit(x_train_scaled,y_train)

KNeighborsRegressor()

In [18]:
print('Train Score', knnr.score(x_train_scaled,y_train))
print('Test Score', knnr.score(x_test_scaled,y_test))

Train Score 0.4011977827781278
Test Score 0.12609409063499843


In [19]:
rfr = RandomForestRegressor()
rfr.fit(x_train_scaled,y_train)

RandomForestRegressor()

In [20]:
print('Train Score', rfr.score(x_train_scaled,y_train))
print('Test Score', rfr.score(x_test_scaled,y_test))

Train Score 0.7178743939287664
Test Score 0.1165692405744887


In [21]:
col_name = list(x_train.columns)

In [22]:
feat_imp = list(rfr.feature_importances_)

In [23]:
feat_imp_df = pd.DataFrame({"col_name":col_name,"feat_imp":feat_imp})

In [24]:
feat_imp_df[['col_name','feat_imp']].sort_values(by='feat_imp')

,col_name,feat_imp
8,type_of_admission_Trauma,0.004735
7,type_of_admission_Not Available,0.007229
6,type_of_admission_Newborn,0.035579
10,apr_medical_surgical_description_Surgical,0.038658
9,type_of_admission_Urgent,0.040973
5,type_of_admission_Emergency,0.048175
4,gender_M,0.083637
3,apr_severity_of_illness_description,0.144772
0,age_group,0.155585
1,ccs_diagnosis_description,0.216228


* as we can see from the above the following features has less than 0.05 importancy we are going to drop them ['type_of_admission_Not Available','type_of_admission_Trauma','type_of_admission_Newborn','apr_medical_surgical_description_Surgical','type_of_admission_Urgent','type_of_admission_Emergency']

In [25]:
# we well drop them from both x_train & x_test
x_train.drop(['type_of_admission_Not Available','type_of_admission_Trauma','type_of_admission_Newborn','apr_medical_surgical_description_Surgical','type_of_admission_Urgent','type_of_admission_Emergency'],axis=1,inplace=True)

In [26]:
x_test.drop(['type_of_admission_Not Available','type_of_admission_Trauma','type_of_admission_Newborn','apr_medical_surgical_description_Surgical','type_of_admission_Urgent','type_of_admission_Emergency'],axis=1,inplace=True)

In [27]:
x_train['length_of_stay'] = y_train

In [28]:
x_train.drop('length_of_stay',axis=1,inplace=True)

In [29]:
x_train.head()

,age_group,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M
178405,2,7.0,10.0,2,0
95869,5,4.0,12.0,4,1
33261,4,4.0,8.0,2,1
108086,4,3.0,7.0,2,1
90571,2,17.0,5.0,3,1


In [30]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [31]:
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)

LinearRegression()

In [32]:
print('Train Score', lr.score(x_train_scaled,y_train))
print('Test Score', lr.score(x_test_scaled,y_test))

Train Score 0.13726359827988732
Test Score 0.13896352122494549


In [33]:
poly = PolynomialFeatures(degree=3)
x_train_poly = poly.fit_transform(x_train_scaled)
x_test_poly = poly.transform(x_test_scaled)

In [34]:
lr = LinearRegression()
lr.fit(x_train_poly,y_train)

LinearRegression()

In [35]:
print('Train Score', lr.score(x_train_poly,y_train))
print('Test Score', lr.score(x_test_poly,y_test))

Train Score 0.17294779848142228
Test Score 0.15604461116329105


In [36]:
rfr = RandomForestRegressor(max_depth=15,max_leaf_nodes=10)
rfr.fit(x_train_poly,y_train)

RandomForestRegressor(max_depth=15, max_leaf_nodes=10)

In [37]:
print('Train Score', rfr.score(x_train_poly,y_train))
print('Test Score', rfr.score(x_test_poly,y_test))

Train Score 0.19186554208865414
Test Score 0.1591127347001695


In [38]:
gnb = GaussianNB()
gnb.fit(x_train_poly,y_train)

GaussianNB()

In [39]:
print('Train Score', gnb.score(x_train_poly,y_train))
print('Test Score', gnb.score(x_test_poly,y_test))

Train Score 0.028643141886903884
Test Score 0.01584567688598002


In [40]:
knnr = KNeighborsRegressor(n_neighbors=11)
knnr.fit(x_train_poly,y_train)

KNeighborsRegressor(n_neighbors=11)

In [41]:
print('Train Score', knnr.score(x_train_poly,y_train))
print('Test Score', knnr.score(x_test_poly,y_test))

Train Score 0.267128705473781
Test Score 0.13960387431702603


In [42]:
# trying different models to compare between them
models = {"Liner Regression": LinearRegression(),
          "KNN": KNeighborsRegressor(n_neighbors=7),
          "SVR": SVR(),
          "NB": GaussianNB(),
          "RF": RandomForestRegressor(n_estimators=100, n_jobs=-1)}

In [43]:
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train_poly,y_train)
    print('Train Score', model.score(x_train_poly,y_train))
    print('Test Score', model.score(x_test_poly,y_test))
    print("-"*60)

-----------  Liner Regression  -------------
Train Score 0.17294779848142228
Test Score 0.15604461116329105
------------------------------------------------------------
-----------  KNN  -------------
Train Score 0.30741477035715015
Test Score 0.11610655016350657
------------------------------------------------------------
-----------  SVR  -------------
Train Score 0.05931143412307083
Test Score 0.046573886461219405
------------------------------------------------------------
-----------  NB  -------------
Train Score 0.028643141886903884
Test Score 0.01584567688598002
------------------------------------------------------------
-----------  RF  -------------
Train Score 0.5209629383020928
Test Score 0.04441048033473871
------------------------------------------------------------


In [44]:
df2 = sample_x.copy()

In [45]:
df2['length_of_stay'] = sample_y 

In [46]:
df2

,age_group,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M,type_of_admission_Emergency,type_of_admission_Newborn,type_of_admission_Not Available,type_of_admission_Trauma,type_of_admission_Urgent,apr_medical_surgical_description_Surgical,length_of_stay
19716,4,6.0,13.0,2,0,1,0,0,0,0,1,5
12180,4,7.0,9.0,4,0,1,0,0,0,0,1,12
86473,2,9.0,9.0,2,1,1,0,0,0,0,0,5
171409,3,4.0,10.0,3,1,1,0,0,0,0,0,28
118522,1,3.0,11.0,4,1,1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
16474,2,6.0,7.0,2,1,0,0,0,0,0,0,1
36304,3,7.0,11.0,3,1,1,0,0,0,0,0,7
126604,2,6.0,6.0,2,0,1,0,0,0,0,0,52
13369,4,7.0,7.0,4,0,1,0,0,0,0,1,6


In [47]:
df2.drop(['type_of_admission_Not Available','type_of_admission_Trauma','type_of_admission_Newborn','apr_medical_surgical_description_Surgical','type_of_admission_Urgent','type_of_admission_Emergency'],axis=1,inplace=True)

In [48]:
df2

,age_group,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M,length_of_stay
19716,4,6.0,13.0,2,0,5
12180,4,7.0,9.0,4,0,12
86473,2,9.0,9.0,2,1,5
171409,3,4.0,10.0,3,1,28
118522,1,3.0,11.0,4,1,3
...,...,...,...,...,...,...
16474,2,6.0,7.0,2,1,1
36304,3,7.0,11.0,3,1,7
126604,2,6.0,6.0,2,0,52
13369,4,7.0,7.0,4,0,6


In [49]:
df2.duplicated().sum()

425

In [50]:
# Drop duplicates and reset the index
df2 = df2.drop_duplicates().reset_index(drop=True)

In [51]:
df2.duplicated().sum()

0

In [52]:
for col in list(df2.columns):
    outliers_indices = detect_outliers(df2, 0, df2.columns)
    col_mean = df2[col].mean()
    df2[col].iloc[outliers_indices] = np.nan


C:\Users\hp\anaconda3\envs\diaapython\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [53]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9251 entries, 0 to 9250
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   age_group                            8094 non-null   float64
 1   ccs_diagnosis_description            8094 non-null   float64
 2   ccs_procedure_description            8474 non-null   float64
 3   apr_severity_of_illness_description  8632 non-null   float64
 4   gender_M                             8632 non-null   float64
 5   length_of_stay                       8632 non-null   float64
dtypes: float64(6)
memory usage: 433.8 KB


In [54]:
df2.describe()

,age_group,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M,length_of_stay
count,8094.000000,8094.000000,8474.000000,8632.000000,8632.000000,8632.000000
mean,3.553126,6.910798,7.731296,2.603452,0.513786,12.427363
std,1.309824,2.935814,3.324376,1.005812,0.499839,10.183785
min,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000
25%,3.000000,5.000000,5.000000,2.000000,0.000000,5.000000
50%,4.000000,6.000000,7.000000,3.000000,1.000000,9.000000
75%,5.000000,9.000000,10.000000,3.000000,1.000000,18.000000
max,5.000000,15.000000,17.000000,4.000000,1.000000,45.000000


In [55]:
# Create a KNN imputer with a specific number of neighbors
imputer = KNNImputer(n_neighbors=2)

In [56]:
# Fit the imputer to your DataFrame and transform the data
imputed_data = imputer.fit_transform(df2)

In [57]:
imputed_df = pd.DataFrame(imputed_data, columns=df2.columns)

In [58]:
imputed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9251 entries, 0 to 9250
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   age_group                            9251 non-null   float64
 1   ccs_diagnosis_description            9251 non-null   float64
 2   ccs_procedure_description            9251 non-null   float64
 3   apr_severity_of_illness_description  9251 non-null   float64
 4   gender_M                             9251 non-null   float64
 5   length_of_stay                       9251 non-null   float64
dtypes: float64(6)
memory usage: 433.8 KB


In [59]:
imputed_df.duplicated().sum()

703

In [60]:
# Drop duplicates and reset the index
imputed_df = imputed_df.drop_duplicates().reset_index(drop=True)

In [61]:
imputed_df.duplicated().sum()

0

In [62]:
imputed_df.describe()

,age_group,ccs_diagnosis_description,ccs_procedure_description,apr_severity_of_illness_description,gender_M,length_of_stay
count,8548.000000,8548.000000,8548.000000,8548.000000,8548.000000,8548.000000
mean,3.549082,6.947638,7.730139,2.603019,0.513046,12.382947
std,1.293378,2.903358,3.302250,1.005428,0.499830,10.149126
min,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000
25%,3.000000,5.000000,5.000000,2.000000,0.000000,5.000000
50%,4.000000,7.000000,7.000000,3.000000,1.000000,9.000000
75%,5.000000,9.000000,10.000000,3.000000,1.000000,18.000000
max,5.000000,15.000000,17.000000,4.000000,1.000000,45.000000


In [63]:
imputed_df.isnull().sum()

age_group                              0
ccs_diagnosis_description              0
ccs_procedure_description              0
apr_severity_of_illness_description    0
gender_M                               0
length_of_stay                         0
dtype: int64

In [64]:
x = imputed_df.drop('length_of_stay',axis=1)
y = imputed_df.length_of_stay

In [65]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [66]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [67]:
# trying different models to compare between them
models = {"Liner Regression": LinearRegression(),
          "KNN": KNeighborsRegressor(n_neighbors=7),
          "SVR": SVR(),
          "RF": RandomForestRegressor(n_estimators=100, n_jobs=-1)}

In [68]:
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train,y_train)
    print('Train Score', model.score(x_train,y_train))
    print('Test Score', model.score(x_test,y_test))
    print("-"*60)

-----------  Liner Regression  -------------
Train Score 0.12984081472280473
Test Score 0.09133073188888541
------------------------------------------------------------
-----------  KNN  -------------
Train Score 0.27924397704931936
Test Score 0.0061541794209674094
------------------------------------------------------------
-----------  SVR  -------------
Train Score 0.10708039422784388
Test Score 0.06100904833336673
------------------------------------------------------------
-----------  RF  -------------
Train Score 0.5054533950414389
Test Score -0.09166278446923393
------------------------------------------------------------


In [69]:
poly2 = PolynomialFeatures(degree=3)
x_train_poly = poly2.fit_transform(x_train)
x_test_poly = poly2.transform(x_test)

In [70]:
# trying different models to compare between them
models = {"Liner Regression": LinearRegression(),
          "KNN": KNeighborsRegressor(n_neighbors=5),
          "RF": RandomForestRegressor()}

In [71]:
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train,y_train)
    print('Train Score', model.score(x_train,y_train))
    print('Test Score', model.score(x_test,y_test))
    print("-"*60)

-----------  Liner Regression  -------------
Train Score 0.12984081472280473
Test Score 0.09133073188888541
------------------------------------------------------------
-----------  KNN  -------------
Train Score 0.3189806605665687
Test Score -0.035452026427420735
------------------------------------------------------------
-----------  RF  -------------
Train Score 0.5051542320695663
Test Score -0.09262536939948518
------------------------------------------------------------
